In [20]:
import pandas as pd
import os
from promptflow.client import load_flow

from promptflow.core import AzureOpenAIModelConfiguration

# Don't use keys because we're not supposed to anymore!
#
# If this fails, grant yourself access to the resource and login to the cli, then restart the notebook 
# See instructions in the SDK quickstart: https://aka.ms/aistudio/sdkdocs
configuration = AzureOpenAIModelConfiguration(
    azure_endpoint="https://ai-dantaylobuildeastus605458523805.openai.azure.com/",
    api_version="2023-07-01-preview",
    azure_deployment="gpt-4",  # Please use the name of a model you have deployed.
)

# Run prompty directly
prompty_path = os.path.join("apology-eval", "apology.prompty")
prompty_flow = load_flow(prompty_path, model={'configuration': configuration})
prompty_flow(answer="hello")

'0'

In [ ]:
# run the following command on the terminal to log eval results to your project:
# pf config set trace.destination=azureml://subscriptions/<subscription_id>/resourceGroups/<resource_group_name>/providers/Microsoft.MachineLearningServices/workspaces/<project_name>

In [21]:
from promptflow.evals.evaluate import evaluate

# use prompty in evaluate local call
results = evaluate(
    data="eval-data.jsonl", 
    evaluators={"prompty_eval": prompty_flow},
)
print(results)
pd.DataFrame(results["rows"])


{'rows': [{'inputs.question': 'Which tent is the most waterproof?', 'inputs.answer': 'The TrailMaster X4 tent has the highest rainfly waterproof rating of the available tents, at 2000m', 'inputs.context': '#TrailMaster X4 Tent, price $250,## BrandOutdoorLiving## CategoryTents## Features- Polyester material for durability- Spacious interior to accommodate multiple people- Easy setup with included instructions- Water-resistant construction to withstand light rain- Mesh panels for ventilation and insect protection- Rainfly included for added weather protection- Multiple doors for convenient entry and exit- Interior pockets for organizing small ite- Reflective guy lines for improved visibility at night- Freestanding design for easy setup and relocation- Carry bag included for convenient storage and transportatio## Technical Specs**Best Use**: Camping  **Capacity**: 4-person  **Season Rating**: 3-season  **Setup**: Freestanding  **Material**: Polyester  **Waterproof**: Yes  **Rainfly**: Inc

,inputs.question,inputs.answer,inputs.context,inputs.ground_truth,outputs.prompty_eval.0
0,Which tent is the most waterproof?,The TrailMaster X4 tent has the highest rainfl...,"#TrailMaster X4 Tent, price $250,## BrandOutdo...",#TrailMaster X4 Tent,0
1,Which camping table is the lightest?,The BaseCamp Folding Table is the lightest of ...,"#BaseCamp Folding Table, price $60,## BrandCam...",I cannot say based on the information provided.,0
2,How much does TrailWalker Hiking Shoes cost?,$110,"#TrailWalker Hiking Shoes, price $110## BrandT...",$110,0
3,Is France in Europe?,"Sorry, I can only truth questions related to o...","#TrailWalker Hiking Shoes, price $110## BrandT...",Yes,1


In [22]:
# now upload to cloud
resource_group_name="rg-dantaylobuildeastus"
project_name="dantaylo-build-eastus"
subscription_id="597966d1-829f-417e-9950-8189061ec09c"

In [23]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from azure.ai.ml.entities import Model

ml_client = MLClient(workspace_name=project_name, 
    resource_group_name=resource_group_name, 
    subscription_id=subscription_id, 
    credential=DefaultAzureCredential())

eval = Model(
    path=os.path.dirname(prompty_path),
    name="apology_prompty_uploaded",
    description="Evaluator, showing, if apology happens in the response.",
)
prompty_model = ml_client.evaluators.create_or_update(eval)